In [1]:
!git clone https://github.com/EleutherAI/clt-training
%cd clt-training

Cloning into 'clt-training'...
remote: Enumerating objects: 2348, done.
remote: Counting objects: 100% (970/970), done.
remote: Compressing objects: 100% (319/319), done.
remote: Total 2348 (delta 833), reused 700 (delta 651), pack-reused 1378 (from 2)
Receiving objects: 100% (2348/2348), 1.05 MiB | 2.62 MiB/s, done.
Resolving deltas: 100% (1669/1669), done.
/content/clt-training


In [2]:
%%file pyproject.toml
[build-system]
requires = ["setuptools"]
build-backend = "setuptools.build_meta"

[project]
name = "eai-sparsify"
description = "Sparsify transformers with SAEs and transcoders"
readme = "README.md"
requires-python = ">=3.10"
keywords = ["interpretability", "explainable-ai"]
license = {text = "MIT License"}
dependencies = [
    "accelerate", # For device_map in from_pretrained
    "datasets",
    "einops",
    "hf-xet>=1.1.0",
    "huggingface-hub",
    "natsort", # For sorting module names
    "safetensors",
    "schedulefree",
    "simple-parsing",
    "torch",
    "transformers",
    "wandb>=0.19.11",
]
version = "1.1.3"
[project.optional-dependencies]
dev = [
    "pre-commit",
]

[project.scripts]
sparsify = "sparsify.__main__:run"

[[tool.uv.index]]
name = "pytorch"
url = "https://download.pytorch.org/whl/cu128"
explicit = true
[tool.uv]
prerelease = "allow"
override-dependencies = [
    "vllm ; sys_platform == 'never'",
    "xformers ; sys_platform == 'never'",
    "torch ; sys_platform == 'never'",
    "triton ; sys_platform == 'never'",
]
[tool.uv.sources]
torch = [
    { index = "pytorch" },
]
triton = [
    { index = "pytorch" }
]

[tool.pyright]
include = ["sparsify*"]
reportPrivateImportUsage = false

[tool.setuptools.packages.find]
include = ["sparsify*"]

[tool.ruff]
exclude = ["scripts/*"]
# Enable pycodestyle (`E`), Pyflakes (`F`), and isort (`I`) codes
# See https://beta.ruff.rs/docs/rules/ for more possible rules
select = ["E", "F", "I"]
# Same as Black.
line-length = 88
# Avoid automatically removing unused imports in __init__.py files.
# Such imports will be flagged with a dedicated message suggesting
# that the import is either added to the module's __all__ symbol
ignore-init-module-imports = true

[tool.semantic_release]
version_variables = ["sparsify/__init__.py:__version__"]
version_toml = ["pyproject.toml:project.version"]
assets = []
build_command_env = []
commit_message = "{version}\n\nAutomatically generated by python-semantic-release"
commit_parser = "conventional"
logging_use_named_masks = false
major_on_zero = true
allow_zero_version = true
no_git_verify = false
tag_format = "v{version}"

[tool.semantic_release.branches.main]
match = "(main|master)"
prerelease_token = "rc"
prerelease = false

[tool.semantic_release.changelog]
changelog_file = ""
exclude_commit_patterns = []
mode = "init"
insertion_flag = "<!-- version list -->"
template_dir = "templates"

[tool.semantic_release.changelog.default_templates]
changelog_file = "CHANGELOG.md"
output_format = "md"
mask_initial_release = false

[tool.semantic_release.changelog.environment]
block_start_string = "{%"
block_end_string = "%}"
variable_start_string = "{{"
variable_end_string = "}}"
comment_start_string = "{#"
comment_end_string = "#}"
trim_blocks = false
lstrip_blocks = false
newline_sequence = "\n"
keep_trailing_newline = false
extensions = []
autoescape = false

[tool.semantic_release.commit_author]
env = "GIT_COMMIT_AUTHOR"
default = "semantic-release <semantic-release>"

[tool.semantic_release.commit_parser_options]
minor_tags = ["feat"]
patch_tags = ["fix", "perf"]
other_allowed_tags = ["build", "chore", "ci", "docs", "style", "refactor", "test"]
allowed_tags = ["feat", "fix", "perf", "build", "chore", "ci", "docs", "style", "refactor", "test"]
default_bump_level = 0
parse_squash_commits = false
ignore_merge_commits = false

[tool.semantic_release.remote]
name = "origin"
type = "github"
ignore_token_for_push = false
insecure = false

[tool.semantic_release.publish]
dist_glob_patterns = ["dist/*"]
upload_to_vcs_release = true

Overwriting pyproject.toml


In [ ]:
!curl -LsSf https://astral.sh/uv/install.sh | sh

downloading uv 0.7.12 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [3]:
!uv pip install -e .

Using Python 3.11.13 environment at: /usr
Resolved 59 packages in 651ms
   Building eai-sparsify @ file:///content/clt-training
   Building eai-sparsify @ file:///content/clt-training
⠙ Preparing packages... (0/2)
   Building eai-sparsify @ file:///content/clt-training
⠙ Preparing packages... (0/2)
   Building eai-sparsify @ file:///content/clt-training
⠙ Preparing packages... (0/2)
   Building eai-sparsify @ file:///content/clt-training
⠙ Preparing packages... (0/2)
   Building eai-sparsify @ file:///content/clt-training
⠙ Preparing packages... (0/2)
   Building eai-sparsify @ file:///content/clt-training
   Building eai-sparsify @ file:///content/clt-training
   Building eai-sparsify @ file:///content/clt-training
   Building eai-sparsify @ file:///content/clt-training
   Building eai-sparsify @ file:///content/clt-training
   Building eai-sparsify @ file:///content/clt-training
      Built eai-sparsify @ file:///content/clt-training
Prepared 2 packages in 1.03s
Installed 2 packages 

In [4]:
%%file train.sh
# python -m sparsify openai-community/gpt2 EleutherAI/fineweb-edu-dedup-10b --ctx_len 128 --filter_bos=True \
python -m sparsify openai-community/gpt2 NeelNanda/pile-10k --ctx_len 128 --filter_bos=True \
--transcode=True --skip_connection=True \
--batch_size=4 \
--hookpoints h.0.mlp h.1.mlp h.2.mlp h.3.mlp h.4.mlp h.5.mlp h.6.mlp h.7.mlp h.8.mlp h.9.mlp h.10.mlp h.11.mlp \
--run_name gpt2-sweep \
--cross_layer=12 --coalesce_topk=concat --topk_coalesced=False \
--post_encoder_scale=True --normalize_io=True \
--lr 3e-4 \
--optimizer adam --lr_warmup_steps 50

Writing train.sh


In [5]:
!uv pip install -U datasets

Using Python 3.11.13 environment at: /usr
Resolved 32 packages in 190ms
⠙ Preparing packages... (0/16)
⠙ Preparing packages... (0/16)
dill       ------------------------------     0 B/113.53 KiB
⠙ Preparing packages... (0/16)
requests   ------------------------------     0 B/63.33 KiB
dill       ------------------------------     0 B/113.53 KiB
⠙ Preparing packages... (0/16)
requests   ------------------------------     0 B/63.33 KiB
dill       ------------------------------ 14.93 KiB/113.53 KiB
⠙ Preparing packages... (0/16)
requests   ------------------------------     0 B/63.33 KiB
dill       ------------------------------ 14.93 KiB/113.53 KiB
⠙ Preparing packages... (0/16)
requests   ------------------------------ 14.88 KiB/63.33 KiB
dill       ------------------------------ 14.93 KiB/113.53 KiB
⠙ Preparing packages... (0/16)
requests   ------------------------------ 14.88 KiB/63.33 KiB
dill       ------------------------------ 30.93 KiB/113.53 KiB
⠙ Preparing packages... (0/16)
re

In [6]:
!bash train.sh

2025-06-16 16:20:29.445940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750090829.464299    2623 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750090829.470266    2623 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-16 16:20:29.490564: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
config.json: 100% 665/665 [00:00<00:00, 4.52MB/s]
model.safetensors: 100% 548M/548M [00:09<00:00, 59.6MB/s]
README.md